<p style="text-align:center;">
<img src="https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true"
     alt="DigitalFuturesLogo"
     style="float: center; margin-right: 10px;" />
</p>

# Digital Futures Python Challenges
### Data Science Python Fundamentals

- Each of these challenges will help you practice and apply your Python Knowledge.     
- Section 1: Brief explanations of relevant tools
- Section 2: Task to test your knowledge.

# 1: Missing Data

One of the things you will have to deal with as a data practitioner is missing data. Whether it arises from incorrect documentation, laziness or systematically it is our job to deal with it.

With pandas data frames, the most common formatting of missing data is as a null value or a numpy nan (not a number) value.

Here we will go through the methods to: 
- identify the nulls and where they are in the data frame,
- how to remove/delete null entries,
- and how to fill in the null values.

In [1]:
# import packages and load data

import pandas as pd
import numpy as np
import seaborn as sns

## Read in the titanic dataframe
df = sns.load_dataset('titanic')

# 1.1 Finding Nulls

There are two very important pandas methods to identify nulls (note: These return truth tables). These are isnull, and notnull. These can be used to identify and separate your data frames by a condition of nulls being present, allowing for efficient comparison.

In [2]:
# counting nulls
null_df = df.isnull() # truth table
null_df.sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [3]:
# counting non nulls
notnull_df = df.notnull() # truth table
notnull_df.sum()

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

In [4]:
# lets identify the amount of nulls and put them into a data frame 
# feel free to copy paste 

def null_vals(dataframe):
    '''function to show both number of nulls and the percentage of nulls in the whole column'''
    null_vals = dataframe.isnull().sum() ## How many nulls in each column
    total_cnt = len(dataframe) ## Total entries in the dataframe
    null_vals = pd.DataFrame(null_vals,columns=['null']) ## Put the number of nulls in a single dataframe
    null_vals['percent'] = round((null_vals['null']/total_cnt)*100,3) ## Round how many nulls are there, as %, of the df
    
    return null_vals.sort_values('percent', ascending=False)

Let's apply it on our Titanic dataframe

In [5]:
null_vals(df)

,null,percent
deck,688,77.217
age,177,19.865
embarked,2,0.224
embark_town,2,0.224
sex,0,0.000
pclass,0,0.000
survived,0,0.000
fare,0,0.000
parch,0,0.000
sibsp,0,0.000


# 1.2: Dropping Nulls

One crude way to handle null values is to simply remove them. This is common practice when there is only a small population of nulls as it likely won't affect the general trends (This assumption should always be verified by investigating the null entries).

Let's drop embarked and embark_town as they are only .224% of the df using the dropna method (Note: there are other solutions)

In [6]:
df.dropna(
    axis = 0, # default
    how = 'any', # default
    subset = ['embarked', 'embark_town'],   # subset tells dropna which columns to consider
    inplace = True # makes change permanent
)

Let's check the results

In [7]:
null_vals(df)

,null,percent
deck,688,77.39
age,177,19.91
pclass,0,0.00
sex,0,0.00
sibsp,0,0.00
parch,0,0.00
survived,0,0.00
fare,0,0.00
embarked,0,0.00
who,0,0.00


No more nulls in embarked and embark_town! Well done :)

# 1.3: Standard Fill Nulls

There are a few ways to fill in missing values using the default fillna method, including:
- ffill/pad - uses last valid value to fill (doesn't work if index 0 is null)
- bfill/backfill - uses next valid value to retroactively fill (doesn't work if last value is null)


or we can fill the data with a predefined value like 0.

In [8]:
# Here we will fill all deck values with 'X' to identify unknown
# This is a justified decision when looking at the distributions of other columns when deck is na
# This is one of our assumptions! Feel free to investigate it

# as the datatype is categorical we need to add X to our categories first.
df['deck'] = df['deck'].cat.add_categories('X')

# only for 1 col
df['deck'].fillna(
    value = 'X', ## our value
    method = None, ## We're using a pre-determined value, not backfilling/padding
    inplace = True ## Have the changes take place
)

C:\Users\ha24a\AppData\Local\Temp\ipykernel_14420\3116131763.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['deck'].fillna(


Again, let's check our results

In [9]:
null_vals(df)

,null,percent
age,177,19.91
pclass,0,0.00
survived,0,0.00
sex,0,0.00
sibsp,0,0.00
parch,0,0.00
fare,0,0.00
embarked,0,0.00
class,0,0.00
who,0,0.00


No more nulls in deck either! Only age left.

# 1.4: Custom Fill Nulls

We can also get more creative with how we fill nulls by introducing the other columns within the data frame.

To fill in the age we could survive with ffill or bfill but with a more sophisticated approach, we can obtain better results when modelling.

What can we use to fill age?

- average age
- average age per gender
- average age per class

additionally, the median, mode, max or min can also be used or even more complex justifications can be used.

Here we will use the average per class however in situations where two numerical columns have high correlation you could even use linear regression to fill in missing values.

In [10]:
# we will use the groupby transform method from pd_04

df['age'] = df.groupby('pclass')[['age']].transform(lambda x: x.fillna(x.mean()))

# check nulls
null_vals(df)

,null,percent
survived,0,0.0
pclass,0,0.0
sex,0,0.0
age,0,0.0
sibsp,0,0.0
parch,0,0.0
fare,0,0.0
embarked,0,0.0
class,0,0.0
who,0,0.0


ALL CLEAN!

interpolation is another useful tool and has a method in [pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate). Feel free to explore this by yourselves if interested.

# 2: Tasks

Here we will work with the mpg dataset from seaborn.

## 2.1: Count Nulls

In [17]:
# load data

mpg = sns.load_dataset('mpg')
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


In [22]:
## TO DO
# count the nulls and see what column they're in
null_count = mpg.isnull().sum()
null_count


mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

## 2.2: Investigate Best Fill Method 

In [13]:
## TO DO
# plot graphs to see distributions and investigate where the nulls occur



## 2.3: [BONUS] Modelling Missingness with Linear Regression

In [14]:
## TO DO
# import and train model
# remember you cant train with nulls
# predict and replace null values



<details> <summary> Click here for a few hints </summary>
    
    1. You can use linear regression with either statsmodels or sklearn.
    
    2. Whichever you choose, your TRAIN set should be all the non-null data. You then predict on the TEST set.
    
    3. Your model's performance can be measured in TRAIN only. Build the best (and most robust!) model on TRAIN, then apply on TEST.

In [15]:
## TO DO
# check there are no nulls and check distribution



Congrats on making it to the end! There is always more to learn, here are a couple of extra resources:

[1. Visualising your missing data with missingno](https://towardsdatascience.com/using-the-missingno-python-library-to-identify-and-visualise-missing-data-prior-to-machine-learning-34c8c5b5f009)

[2. Using interpolation & central tendencies](https://pythonsansar.com/how-to-handle-null-values-in-pandas/)

[3. Why is my data missing??](https://www.scribbr.com/statistics/missing-data/)

[4. Modelling your missingness](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)


**Hope you enjoyed!**